In [1]:
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from sklearn.metrics import confusion_matrix
import multiprocessing
import copy
from tqdm import tqdm
import time
from math import sqrt
from skimage import io 
from PIL import Image, ImageFilter
import random
from keras.preprocessing.image import ImageDataGenerator 
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Model Testing



In this section we will be testing the CNN as defined in section 4 under two different modifications to its input. Firstly, we will investigate the effect of the image pre-processing in section 3 by training an initial CNN (model) on the unprocessed data, and an alternative CNN (new_model) on the new, augmented data. 

Secondly, we will attempt to test our hypothesis as proposed in section 4: that the angle that the image is taken from is an important factor in the performance of the model. To test this, we will attempt to classify the test set into three categories: 'Side-On', 'Back', and 'Top-Down'. Then we will run the CNN on each category of data to see if there is any noticable difference in prediction accuracy. 

We begin by loading in both the original and preprocessed training data, and the original test data.

*Note: We will be loading in a lot of data and training multiple neural networks in this section, so to conserve memory we now resize the images to 100x100.*

In [2]:
# LOADING THE DATA

#Defining our four class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

#No. of classes
nb_classes = len(class_names)

#Image pixel width x height
IMAGE_SIZE = (100, 100)

#Loading Training and Testing folders
datasets = ['Data/Training','NewData/PreProcessed_Data/Training','Data/Testing']
output = []
#Perform for both Training and Testing
for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        #Searching through each folder corresponding to a class
        for folder in class_names:
            label = class_names_label[folder]
            
            #Searching through every image within the folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Getting path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Opening and resizing image
                image = cv2.imread(img_path)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its label to the output
                images.append(image)
                labels.append(label)
        
        #Convert to array
        images = np.array(images)
        labels = np.array(labels)   
        
        output.append((images, labels))

#Define training and testing images and labels
#(train_images, train_labels) = output
#train_images = output[0][0]
#train_labels = output[0][1]
(train_images, train_labels), (new_train_images, new_train_labels), (test_images, test_labels) = output

Loading /kaggle/input/brain-tumor-mri-dataset/Training


100%|██████████| 1457/1457 [00:10<00:00, 140.95it/s]


Loading /kaggle/input/processed-brain-data/Training


100%|██████████| 5893/5893 [00:23<00:00, 246.58it/s]


Loading /kaggle/input/brain-tumor-mri-dataset/Testing


100%|██████████| 300/300 [00:02<00:00, 123.81it/s]


Then to label the angles of the testing data, we use the images outputted by section 6

In [3]:
# LOADING THE DATA

#Defining our four class names
class_names = ['back', 'sideon', 'topdown']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

#No. of classes
nb_classes = len(class_names)

#Image pixel width x height
IMAGE_SIZE = (100, 100)

#Loading Training and Testing folders
datasets = ['NewData/Angle_Data']
output = []
#Perform for both Training and Testing
for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        #Searching through each folder corresponding to a class
        for folder in class_names:
            label = class_names_label[folder]
            
            #Searching through every image within the folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Getting path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Opening and resizing image
                image = cv2.imread(img_path)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its label to the output
                images.append(image)
                labels.append(label)
        
        #Convert to array
        images = np.array(images)
        labels = np.array(labels)   
        
        output.append((images, labels))

#Define training and testing images and labels
#(train_images, train_labels) = output
angle_train_images = output[0][0]
angle_train_labels = output[0][1]

Loading /kaggle/input/shreyas-weird-images/Train and Output Combined


100%|██████████| 9041/9041 [00:40<00:00, 225.17it/s]


We begin by defining both our CNNs in exactly the same way, as in section 4, and then train one off the initial data and one off the preproccesed data.

In [4]:
#BUILDING INITIAL MODEL
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25), 
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),  
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax)
])

#Compiling model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
#TRAINING CNN
model.fit(train_images, train_labels, batch_size=32, epochs=20, validation_split = 0.2)

Epoch 1/20


2023-05-03 08:22:06.401432: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


143/143 [==============================] - 9s 12ms/step - loss: 12.7391 - accuracy: 0.5060 - val_loss: 1.3312 - val_accuracy: 0.3115
Epoch 2/20
143/143 [==============================] - 1s 8ms/step - loss: 0.8319 - accuracy: 0.6345 - val_loss: 1.2963 - val_accuracy: 0.4707
Epoch 3/20
143/143 [==============================] - 1s 8ms/step - loss: 0.6439 - accuracy: 0.7380 - val_loss: 1.1561 - val_accuracy: 0.6710
Epoch 4/20
143/143 [==============================] - 1s 8ms/step - loss: 0.5251 - accuracy: 0.7914 - val_loss: 1.1249 - val_accuracy: 0.5766
Epoch 5/20
143/143 [==============================] - 1s 8ms/step - loss: 0.4527 - accuracy: 0.8203 - val_loss: 1.0132 - val_accuracy: 0.6763
Epoch 6/20
143/143 [==============================] - 1s 8ms/step - loss: 0.4068 - accuracy: 0.8352 - val_loss: 1.0179 - val_accuracy: 0.6850
Epoch 7/20
143/143 [==============================] - 1s 8ms/step - loss: 0.3878 - accuracy: 0.8545 - val_loss: 1.0529 - val_accuracy: 0.6535
Epoch 8/20
143/

In [6]:
#BUILDING INITIAL MODEL
new_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25), 
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),  
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax)
])

#Compiling model
new_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
#TRAINING CNN
new_model.fit(new_train_images, new_train_labels, batch_size=55, epochs=20, validation_split = 0.2)

Epoch 1/20


2023-05-03 08:22:49.206717: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_3/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


336/336 [==============================] - 6s 13ms/step - loss: 6.5630 - accuracy: 0.5522 - val_loss: 1.7219 - val_accuracy: 0.0396
Epoch 2/20
336/336 [==============================] - 4s 11ms/step - loss: 0.7054 - accuracy: 0.7084 - val_loss: 1.6052 - val_accuracy: 0.2326
Epoch 3/20
336/336 [==============================] - 4s 11ms/step - loss: 0.6107 - accuracy: 0.7442 - val_loss: 1.4742 - val_accuracy: 0.4244
Epoch 4/20
336/336 [==============================] - 4s 11ms/step - loss: 0.5659 - accuracy: 0.7669 - val_loss: 1.5135 - val_accuracy: 0.3971
Epoch 5/20
336/336 [==============================] - 4s 12ms/step - loss: 0.5197 - accuracy: 0.7859 - val_loss: 1.3209 - val_accuracy: 0.5522
Epoch 6/20
336/336 [==============================] - 4s 11ms/step - loss: 0.4833 - accuracy: 0.8051 - val_loss: 1.1995 - val_accuracy: 0.5671
Epoch 7/20
336/336 [==============================] - 3s 10ms/step - loss: 0.4516 - accuracy: 0.8156 - val_loss: 1.2517 - val_accuracy: 0.5628
Epoch 8/20

### Angle Prediction

Then as a method for labelling the angles of the test data, we will first train a CNN model on the data from section 6, where each image is sorted into a folder by angle. We then use this trained angle model to predict the angle of each test image, and then sort the test images into the 3 angles using this prediction.

In [8]:
#BUILDING INITIAL MODEL
angle_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25), 
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),  
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

#Compiling model
angle_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
#TRAINING CNN
angle_model.fit(angle_train_images, angle_train_labels, batch_size=55, epochs=20, validation_split = 0.2)

Epoch 1/20


2023-05-03 08:24:03.565242: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


267/267 [==============================] - 5s 13ms/step - loss: 7.0354 - accuracy: 0.7507 - val_loss: 0.2454 - val_accuracy: 0.8994
Epoch 2/20
267/267 [==============================] - 3s 11ms/step - loss: 0.3709 - accuracy: 0.8693 - val_loss: 0.2113 - val_accuracy: 0.9076
Epoch 3/20
267/267 [==============================] - 3s 12ms/step - loss: 0.3101 - accuracy: 0.8884 - val_loss: 0.2425 - val_accuracy: 0.9008
Epoch 4/20
267/267 [==============================] - 3s 11ms/step - loss: 0.2879 - accuracy: 0.8955 - val_loss: 0.2212 - val_accuracy: 0.9090
Epoch 5/20
267/267 [==============================] - 3s 11ms/step - loss: 0.2618 - accuracy: 0.9045 - val_loss: 0.2447 - val_accuracy: 0.9030
Epoch 6/20
267/267 [==============================] - 3s 11ms/step - loss: 0.2425 - accuracy: 0.9104 - val_loss: 0.2417 - val_accuracy: 0.9002
Epoch 7/20
267/267 [==============================] - 3s 11ms/step - loss: 0.2327 - accuracy: 0.9147 - val_loss: 0.2417 - val_accuracy: 0.9041
Epoch 8/20

In [10]:
test_angle_predictions = angle_model.predict(test_images)
test_angle_pred_labels = np.argmax(test_angle_predictions, axis =1)

41/41 [==============================] - 0s 3ms/step


In [11]:
test_sideon_images=[]
test_sideon_labels=[]

test_topdown_images=[]
test_topdown_labels=[]

test_back_images=[]
test_back_labels=[]

for i in range(len(test_images)):
    if (test_angle_pred_labels[i]==0):
        test_back_images.append(test_images[i])
        test_back_labels.append(test_labels[i])
    if (test_angle_pred_labels[i]==1):
        test_sideon_images.append(test_images[i])
        test_sideon_labels.append(test_labels[i])
    if (test_angle_pred_labels[i]==2):
        test_topdown_images.append(test_images[i])
        test_topdown_labels.append(test_labels[i])
        

### Predictive Accuracy

Finally we are in a position to evaluate the performance of each of these models. First we calculate the accuracy of both 'model' and 'new_model' on the entire test data. Here we can see that the accuracy is improved by the new_model, suggesting that augmenting the images as in the preprocessing stage helped the model train better in differentiating between the image classes.

In [12]:
#CALCULATING TEST PREDICTION
test_loss = model.evaluate(test_images, test_labels)
new_test_loss = new_model.evaluate(test_images, test_labels)

41/41 [==============================] - 0s 3ms/step - loss: 0.1852 - accuracy: 0.9405


Next we evaluate the new model on the test data separated by angle. As we can see, the predictive accuracy given below seems to give evidence to our hypothesis that there is a great disparity between the performance of the model on 'Top-Down' images relative to any other image.

Whilst the lack of interpretability in the CNN makes it difficult to pinpoint exactly why this may be, an intuitive suggestion is the same one as in section 4: that the 'Top-Down' images contain less artifacts within the images. For instance, most of the top down images, especially in the 'notumour' folder, are simply a relatively undetailed grey circle for the brain with a very identifiable white circle surrounding it. However for the 'Side-On' and 'Back' images, not only is the overall shape of the head inconsistent, we now have to deal with other objects such as noses, eye sockets, jawbones and teeth etc. 

In [13]:
sideon_test_loss = new_model.evaluate(np.array(test_sideon_images), np.array(test_sideon_labels))
topdown_test_loss = new_model.evaluate(np.array(test_topdown_images), np.array(test_topdown_labels))
back_test_loss = new_model.evaluate(np.array(test_back_images), np.array(test_back_labels))

9/9 [==============================] - 0s 5ms/step - loss: 0.2626 - accuracy: 0.9225


To conclude then, we can have seen both the image augmentation modification and the identifying of image angle both have a positive effect on the performance of our neural network. 

As an extension, we could look at how to best mitigate the problem of differences in performance between angles. So far we have shown the problem exists, and we can give different levels of uncertainty to a prediction based on its angle, but no actual solution of how to improve the model. To do this we suggest predicting the angle of every image in the training set using the angle CNN (angle_model), and somehow incorporating these new labels into the CNN to retrain the model on both the images and angle labels simultaneously.

### New Data

Finally, we can test the generalisation of our model by introducing completely new data from an external dataset: https://radiopaedia.org/?lang=gb

Here the predictive accuracy is quite poor, although we have only used less than 100 new images so it is unclear if we can make any proper inference from this. It is likely though that although the model has been generalised due to the augmented data, these new images still need to be properly preprocessed before being inputted.


In [14]:
# LOADING THE DATA

#Defining our four class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

#No. of classes
nb_classes = len(class_names)

#Image pixel width x height
IMAGE_SIZE = (100, 100)

#Loading Training and Testing folders
datasets = ['NewData/New_MRI_Images']
output = []
#Perform for both Training and Testing
for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        #Searching through each folder corresponding to a class
        for folder in class_names:
            label = class_names_label[folder]
            
            #Searching through every image within the folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Getting path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Opening and resizing image
                image = cv2.imread(img_path)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its label to the output
                images.append(image)
                labels.append(label)
        
        #Convert to array
        images = np.array(images)
        labels = np.array(labels)   
        
        output.append((images, labels))

        
new_test_images = output[0][0]
new_test_labels = output[0][1]

Loading /kaggle/input/newnewbraindata/MRI Brain Scans


100%|██████████| 20/20 [00:00<00:00, 46.39it/s]


In [15]:
new_new_test_loss = new_model.evaluate(new_test_images, new_test_labels)

3/3 [==============================] - 0s 14ms/step - loss: 7.1078 - accuracy: 0.4568
